# 1. Logistic Regression

In [1]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

from sqlalchemy import create_engine
# from config import db_password
from sqlalchemy import inspect
import matplotlib.pyplot as plt
import pandas as pd

business_df = pd.read_csv("../../../Data/business_census_merged_dataset.csv")
#business_df = pd.read_csv("../../../Data/merged_datasets.csv")

# Categorizing restaurants based on stars ratings
business_df["Category"] = pd.cut(business_df["Stars_Rating"],bins=[0.9,3.5,5],
                                 labels=["Lower Performance", "Higher Performance"])

def changeStatus(status):
    if status == "Lower Performance":
        return 0
    else:
        return 1
    
business_df['Category_Encoded'] = business_df["Category"].apply(changeStatus)
business_df["Category_Encoded"] = pd.to_numeric(business_df["Category_Encoded"])

business_df["Median_Income(dollars)"] = pd.to_numeric(business_df["Median_Income(dollars)"], errors='coerce')
business_df["Mean_Income(dollars)"] = pd.to_numeric(business_df["Mean_Income(dollars)"], errors='coerce')

business_df = business_df.dropna(subset=['Median_Income(dollars)'])

In [2]:
business_df["Category"].value_counts()

Higher Performance    12794
Lower Performance     11945
Name: Category, dtype: int64

In [3]:
business_df["Stars_Rating"].describe()

count    24739.000000
mean         3.665811
std          0.694513
min          1.000000
25%          3.500000
50%          4.000000
75%          4.000000
max          5.000000
Name: Stars_Rating, dtype: float64

In [4]:
business_df.columns

Index(['Restaurant_ID', 'Restaurants_Name', 'Address', 'City', 'State',
       'Postal_Code', 'Latitude', 'Longitude', 'Stars_Rating', 'Review_Count',
       'Restaurants_Delivery', 'Outdoor_Seating', 'Accepts_CreditCards',
       'Price_Range', 'Alcohol', 'Good_For_Kids', 'Reservations',
       'Restaurants_TakeOut', 'WiFi', 'Good_For_Groups',
       'Wheelchair_Accessible', 'Happy_Hour', 'Noise_Level',
       'Dietary_Restrictions', 'Total_Estimate_Households_per_Zip',
       'Total_Estimate_Married-couple_Family_households',
       'Total_Estimate_Nonfamily_households', 'Median_Income(dollars)',
       'Mean_Income(dollars)', 'Category', 'Category_Encoded'],
      dtype='object')

In [6]:
# Define features set
X = business_df[['Review_Count', 'Restaurants_Delivery', 'Outdoor_Seating',
       'Accepts_CreditCards', 'Price_Range', 'Alcohol', 'Good_For_Kids',
       'Reservations', 'Restaurants_TakeOut', 'WiFi', 'Good_For_Groups',
       'Wheelchair_Accessible', 'Happy_Hour', 'Noise_Level',
       'Dietary_Restrictions',
                'Total_Estimate_Married-couple_Family_households',
       'Total_Estimate_Nonfamily_households',
                'Median_Income(dollars)', 'Total_Estimate_Households_per_Zip']]

In [7]:
# Define the target
y = business_df["Category_Encoded"]
y
from collections import Counter
Counter(y)

Counter({1: 12794, 0: 11945})

In [8]:
# Split the model into training and testing sets
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X,
                                                   y, 
                                                   random_state=1, 
                                                    stratify=y)

In [9]:
# Scale the Training and Testing Data
from sklearn.preprocessing import StandardScaler

# Instantiate a StandardScaler()
scaler = StandardScaler()

# Fitting the Standard Scaler with the training data
X_scaler = scaler.fit(X_train)


# Scaling the data 
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [10]:
# Instantiate a Logistic Regression Model
from sklearn.linear_model import LogisticRegression

model = LogisticRegression(solver="lbfgs", random_state=1)
model

LogisticRegression(random_state=1)

In [11]:
# Train the model
model.fit(X_train, y_train)

LogisticRegression(random_state=1)

In [12]:
# Make predictions
y_pred = model.predict(X_test)

In [13]:
# Evaluate the Model
from sklearn.metrics import accuracy_score
accuracy_score(y_test, y_pred)

0.5734842360549717

In [17]:
y_pred_train = model.predict(X_train)
accuracy_score(y_train, y_pred_train)

0.5866120513096906

In [18]:
# Calculate the confusion matrix 
from sklearn.metrics import confusion_matrix, classification_report

matrix=confusion_matrix(y_test, y_pred)
print(matrix)

[[1872 1114]
 [1524 1675]]


In [19]:
report=classification_report(y_test, y_pred)
print(report)

              precision    recall  f1-score   support

           0       0.55      0.63      0.59      2986
           1       0.60      0.52      0.56      3199

    accuracy                           0.57      6185
   macro avg       0.58      0.58      0.57      6185
weighted avg       0.58      0.57      0.57      6185

